In [ ]:
# ============================================================================
# Cell 1: Environment Setup (Run this first, then restart runtime manually)
# ============================================================================
import sys
import subprocess

print("Fixing package conflicts and installing CPU-only versions...")

def fix_environment():
    packages_to_remove = ["torch", "torchvision", "torchaudio", "transformers", "accelerate"]
    for pkg in packages_to_remove:
        subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", pkg],
                       capture_output=True, check=False)

    subprocess.check_call([
        sys.executable, "-m", "pip", "install",
        "torch==2.0.1+cpu", "torchvision==0.15.2+cpu", "torchaudio==2.0.2+cpu",
        "--index-url", "https://download.pytorch.org/whl/cpu", "--quiet", "--force-reinstall"
    ])

    subprocess.check_call([
        sys.executable, "-m", "pip", "install",
        "transformers==4.30.2", "--no-deps", "--quiet"
    ])

    other_packages = [
        "tokenizers>=0.13.0", "numpy", "packaging", "pyyaml", "regex", "requests",
        "safetensors", "tqdm>=4.27", "huggingface-hub>=0.14.0", "datasets==2.12.0",
        "peft==0.4.0", "accelerate==0.20.3"
    ]
    for pkg in other_packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "--quiet"])

fix_environment()
print("\n✅ Environment setup completed!")

print("\n" + "="*60)
print("🚨 RESTART RUNTIME NOW: Runtime > Restart runtime")
print("Then continue from the next cell.")
print("="*60)


Fixing package conflicts and installing CPU-only versions...

✅ Environment setup completed!

🚨 RESTART RUNTIME NOW: Runtime > Restart runtime
Then continue from the next cell.


In [ ]:
# ============================================================================
# Cell 2: Basic Imports After Restart
# ============================================================================
import warnings
warnings.filterwarnings("ignore")

import torch
import transformers
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

print(f"✓ PyTorch {torch.__version__} loaded")
print(f"✓ Transformers {transformers.__version__} loaded")
print("✓ Model and dataset modules imported")
print(f"Device: CPU (CUDA available: {torch.cuda.is_available()})")


✓ PyTorch 2.0.1+cpu loaded
✓ Transformers 4.30.2 loaded
✓ Model and dataset modules imported
Device: CPU (CUDA available: False)


In [ ]:
# ============================================================================
# Cell 3: Create Sample Dataset
# ============================================================================
print("Creating training dataset...")

training_data = [
    "Human: Hello! Assistant: Hi there! How can I help you today?",
    "Human: What is AI? Assistant: AI stands for Artificial Intelligence.",
    "Human: How are you? Assistant: I'm doing well, thank you for asking!",
    "Human: Tell me a fact. Assistant: The Earth orbits around the Sun.",
    "Human: What's 2+2? Assistant: 2+2 equals 4.",
    "Human: Good morning! Assistant: Good morning! Hope you have a great day!",
    "Human: What is Python? Assistant: Python is a programming language.",
    "Human: Help me learn. Assistant: I'd be happy to help you learn!",
    "Human: Thank you. Assistant: You're very welcome!",
    "Human: Goodbye! Assistant: Goodbye! Have a wonderful day!",
] * 5

dataset = Dataset.from_dict({"text": training_data})
print(f"✓ Dataset created with {len(dataset)} samples")


Creating training dataset...
✓ Dataset created with 50 samples


In [ ]:
# ============================================================================
# Cell 4: Load Pretrained Model & Tokenizer
# ============================================================================
print("Loading DistilGPT-2 model...")

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained("distilgpt2")
print(f"✓ Model loaded with {sum(p.numel() for p in model.parameters()):,} parameters")


Loading DistilGPT-2 model...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Using pad_token, but it is not set yet.


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✓ Model loaded with 81,912,576 parameters


In [ ]:
# ============================================================================
# Cell 5: Tokenize Data & Create Dataset Loader
# ============================================================================
from torch.utils.data import DataLoader

def tokenize_data(texts, tokenizer, max_length=64):
    tokenized = []
    for text in texts:
        tokens = tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        tokenized.append({
            'input_ids': tokens['input_ids'].squeeze(0),
            'attention_mask': tokens['attention_mask'].squeeze(0)
        })
    return tokenized

print("Tokenizing data...")
tokenized_data = tokenize_data(training_data, tokenizer)

class SimpleDataset:
    def __init__(self, tokenized_data):
        self.data = tokenized_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            'input_ids': item['input_ids'],
            'attention_mask': item['attention_mask'],
            'labels': item['input_ids'].clone()
        }

train_dataset = SimpleDataset(tokenized_data)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
print("✓ Tokenized dataset ready")


Tokenizing data...
✓ Tokenized dataset ready


In [ ]:
# ============================================================================
# Cell 6: Fine-Tuning Loop (Small & Simple)
# ============================================================================
from torch.optim import AdamW

print("Starting training...")
optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()

num_epochs = 5
max_steps = 100
step_count = 0

for epoch in range(num_epochs):
    for batch in train_loader:
        if step_count >= max_steps:
            break

        outputs = model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            labels=batch['labels']
        )

        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        step_count += 1

        if step_count % 5 == 0:
            print(f"Step {step_count}/{max_steps}, Loss: {loss.item():.4f}")

print("✓ Training completed")


Starting training...
Step 5/100, Loss: 0.7410
Step 10/100, Loss: 0.8570
Step 15/100, Loss: 0.3261
Step 20/100, Loss: 0.3694
Step 25/100, Loss: 0.2372
Step 30/100, Loss: 0.2575
Step 35/100, Loss: 0.3666
Step 40/100, Loss: 0.1493
Step 45/100, Loss: 0.1031
Step 50/100, Loss: 0.1624
Step 55/100, Loss: 0.0865
Step 60/100, Loss: 0.0741
Step 65/100, Loss: 0.0540
Step 70/100, Loss: 0.0844
Step 75/100, Loss: 0.1078
Step 80/100, Loss: 0.0640
Step 85/100, Loss: 0.1547
Step 90/100, Loss: 0.1692
Step 95/100, Loss: 0.0634
Step 100/100, Loss: 0.0967
✓ Training completed


In [ ]:
# ============================================================================
# Cell 8: Test Inference from Fine-Tuned Model
# ============================================================================
model.eval()

def generate_response(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=inputs['input_ids'].shape[1] + max_length,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

test_prompts = [
    "Human: Hello!",
    "Human: What is AI?",
    "Human: How are you?"
]

print("\nGenerated responses:")
print("-" * 40)
for prompt in test_prompts:
    response = generate_response(prompt)
    print(f"Input: {prompt}")
    print(f"Output: {response}")
    print("-" * 40)



Generated responses:
----------------------------------------
Input: Human: Hello!
Output: Human: Hello! Assistant: Hi there! How can I help you today?
----------------------------------------
Input: Human: What is AI?
Output: Human: What is AI? Assistant: AI stands for Artificial Intelligence.
----------------------------------------
Input: Human: How are you?
Output: Human: How are you? Assistant: I'm doing well, thank you for asking!
----------------------------------------
